In [1]:
import re
import base64
from openai import OpenAI
from anthropic import Anthropic
from pdf2image import convert_from_path

from pathlib import Path
from tqdm import tqdm
import pandas as pd
import json

import os
os.getcwd()

'/Users/roshansk/Documents/GitHub/Global_Exams'

In [2]:
def encode_image(image_path):
    """
    Function to encode the image
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def get_message_list(base64_string, prompt):

    message_list = [
    {
        "role": 'user',
        "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": base64_string}},
            {"type": "text", "text": prompt}
        ]
    }
    ]

    return message_list

In [3]:
from anthropic import Anthropic

client = Anthropic()

# MODEL_NAME = "claude-3-opus-20240229"

MODEL_NAME = "claude-3-5-sonnet-20240620"




In [46]:
language = 'hindi'

pdf_file = '/Users/roshansk/Documents/GitHub/Global_Exams/data/2020/NEET_2020_Gujarati_F6.pdf'
answer_file = '/Users/roshansk/Documents/GitHub/Global_Exams/data/2020/NEET_2020_F6_Answer_Key.json'
image_folder = '/Users/roshansk/Documents/GitHub/Global_Exams/images/NEET_2020_Gujarati_F6'


lang_code = 'gu'
country = 'India'
file_name = 'Paper_20201106083723.pdf'
source = 'https://www.nta.ac.in/Download/ExamPaper/Paper_20201106083723.pdf'
license = 'open'
level = 'University'

output_file_name = 'NEET_2020_Gujarati_processed'

pages_to_include=[26,45]

In [11]:
# prompt = """Extract the multiple choice questions along with the options present in the image. The text is mostly in the language of {lang}.
# Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
# The json value should have the following keys : 
# - number : The number of the question
# - question : The actual text of the question
# - options : A list containing all 4 options for the question
# - image : output True if there is an image associated with the either the question or answer and the student is supposed to use an image to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

# Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python directly using eval(response.text)
# The questions are present in 2 columns. Make sure to extract questions from both columns and not just one

# """

# prompt_hindi = """You are given a pdf containing Hindi and English questions. Extract the Hindi multiple choice questions along with the options present in the image.
# Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
# The json value should have the following keys : 
# - number : The number of the question
# - question : The actual text of the question
# - options : A list containing all 4 options for the question
# - image : output True if there is an image associated with the either the question or answer and the student is supposed to use an image to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

# Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python directly using eval(response.text)
# The questions are present in two columns with Hindi questions on the left column and English questions on the right column. Extract only the Hindi questions from the left column.
# """


# prompt = """You are given a pdf containing Gujarati and English questions. Extract the Gujarati multiple choice questions along with the options present in the image.
# Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
# The json value should have the following keys : 
# - number : The number of the question
# - question : The actual text of the question
# - options : A list containing all 4 options for the question
# - image : output True if there is an image associated with the either the question or answer and the student is supposed to use an image to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

# Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python dict directly using eval(response.text)
# The questions are present in two columns with Gujarati questions on the right side and English questions on the left. Extract only the Gujarati questions from the right column.
# """

prompt = """Extract the multiple choice questions along with the options present in the image. The text is mostly in the language of Gujarati.
Return the extracted the questions as a list of json values, with each json value corresponding to a single question. 
The json value should have the following keys : 
- number : The number of the question
- question : The actual text of the question
- options : A list containing all 4 options for the question
- image : output True if there is an image or table associated with the either the question or answer and the student is supposed to use an image/table to answer the question. Output False, otherwise. Ensure that True and False have their first letters capitalized.

Output the list alone and no other supporting text in your response. Ensure that the output can be converted into a python dict directly using eval(response.text)
The questions are present in 2 columns. Make sure to extract questions from both columns and not just one

"""

## Creating Image Files

In [5]:
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

pages = convert_from_path(pdf_file)

start_page, end_page = pages_to_include 
count = 0
for i in range(start_page-1,  end_page):
    filename = "page_"+str(i)+'.jpg'
    pages[i].save(Path(image_folder) / filename)
    count += 1
    

In [9]:
print(f"{count} images created")

## Extracting questions from Claude

In [12]:
imageList = os.listdir(image_folder)

json_output = {}
text_output = {}

for i in tqdm(range(len(imageList))):
    image_path = Path(image_folder) / imageList[i]
    base64_string = encode_image(image_path)

    message_list = get_message_list(base64_string, prompt) 

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        messages=message_list,
        temperature = 0.0,
        top_p = 1
        
    )

    try:
        out = eval(response.content[0].text)

        json_output[i] = out 
        print(f"{imageList[i]} added to json")
    except:
        text_output[i] = response.content[0].text
        print(f"{imageList[i]} added to text")
    
json_backup = json_output.copy()


  5%|██▏                                         | 1/20 [00:29<09:23, 29.63s/it]

page_44.jpg added to json


 10%|████▍                                       | 2/20 [00:55<08:15, 27.52s/it]

page_40.jpg added to json


 15%|██████▌                                     | 3/20 [01:16<06:53, 24.32s/it]

...

page_39.jpg added to json


 95%|████████████████████████████████████████▊  | 19/20 [08:40<00:29, 29.05s/it]

page_29.jpg added to json


100%|███████████████████████████████████████████| 20/20 [08:57<00:00, 26.90s/it]

page_28.jpg added to json


In [38]:
output = []
for key in json_output.keys():
    output += json_output[key]


df = pd.DataFrame(output)
df.columns = ['original_question_num','question','options','image']

print(f"Total no of samples : {len(df)}")
print(f"Total no of samples without images: {len(df[df.image==False])}")
df.head()

Total no of samples : 180
Total no of samples without images: 161


,original_question_num,question,options,image
0,170,એક અવરોધક માટે વર્ણ-સંકેત નીચે આપેલ છે :,"[4.7 kΩ, 5%, 470 Ω, 5%, 470 kΩ, 5%, 47 kΩ, 10%]",True
1,171,r-ત્રિજ્યા ધરાવતી એક કેશનળી ટ્યુબ (કેપિલરી) ને...,"[10.0 g, 20.0 g, 2.5 g, 5.0 g]",False
2,172,એક વાયુકોષમાં 249 kPa દબાણે અને 27°C તાપમાને હ...,"[0.1 kg/m³, 0.02 kg/m³, 0.5 kg/m³, 0.2 kg/m³]",False
3,173,અવરોધના ગુણ તાપમાન ગુણાંક ધરાવતા હોય તેવા 'ધન ...,"[ફક્ત અર્ધવાહકો, અવાહકો અને અર્ધવાહકો, ધાતુઓ, ...",False
4,174,એક એકપરમાણવીય વાયુની સરેરાશ ઉષ્મા ઊર્જા છે ___...,"[5/2 kBT, 7/2 kBT, 1/2 kBT, 3/2 kBT]",False


#### Removing samples with images/tables

In [39]:
df = df[df.image == False]

#### Attaching Answer Key

In [40]:
with open(answer_file) as f:
    answer_key = json.load(f)

In [41]:
df.original_question_num = df.original_question_num.apply(lambda x: x.replace(".","")) #Cleaning

df['answer'] = df.original_question_num.apply(lambda x : str(answer_key[x]))

#### Assigning categories

In [42]:
category_map = {'biology': 'जीवविज्ञान' ,'chemistry':'रसायन विज्ञान','physics':'भौतिक विज्ञान'}

question_category_map = {}
for i in range(1,46):
    question_category_map[i] = 'physics'

for i in range(46,91):
    question_category_map[i] = 'chemistry'

for i in range(91,181):
    question_category_map[i] = 'biology'

In [43]:
df['category_en'] = df.original_question_num.apply(lambda x : question_category_map[int(x)])
df['category_original_lang'] = df.category_en.apply(lambda x : category_map[x])

#### Assigning other metadata

In [44]:
df['language'] = lang_code
df['country'] = country
df['file_name'] = file_name
df['source'] = source
df['license'] = license
df['level'] = level


df_ = df.copy()
df.drop('image', axis = 1, inplace = True)

cols = ['language',
       'country', 'file_name', 'source', 'license', 'level', 'category_en',
       'category_original_lang', 'original_question_num', 'question', 'options', 'answer']

df = df[cols]

In [45]:
df.head()

,language,country,file_name,source,license,level,category_en,category_original_lang,original_question_num,question,options,answer
1,gu,India,Paper_20201106083723.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवविज्ञान,171,r-ત્રિજ્યા ધરાવતી એક કેશનળી ટ્યુબ (કેપિલરી) ને...,"[10.0 g, 20.0 g, 2.5 g, 5.0 g]",1
2,gu,India,Paper_20201106083723.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवविज्ञान,172,એક વાયુકોષમાં 249 kPa દબાણે અને 27°C તાપમાને હ...,"[0.1 kg/m³, 0.02 kg/m³, 0.5 kg/m³, 0.2 kg/m³]",4
3,gu,India,Paper_20201106083723.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवविज्ञान,173,અવરોધના ગુણ તાપમાન ગુણાંક ધરાવતા હોય તેવા 'ધન ...,"[ફક્ત અર્ધવાહકો, અવાહકો અને અર્ધવાહકો, ધાતુઓ, ...",2
4,gu,India,Paper_20201106083723.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवविज्ञान,174,એક એકપરમાણવીય વાયુની સરેરાશ ઉષ્મા ઊર્જા છે ___...,"[5/2 kBT, 7/2 kBT, 1/2 kBT, 3/2 kBT]",4
5,gu,India,Paper_20201106083723.pdf,https://www.nta.ac.in/Download/ExamPaper/Paper...,open,University,biology,जीवविज्ञान,175,20 cm² ક્ષેત્રફળ ધરાવતી એક અખરાવતિત સપાટી પર 2...,"[24×10³ J, 48×10³ J, 10×10³ J, 12×10³ J]",1


#### Save processed file

In [47]:
df.to_json(output_file_name+'.json', orient='records')
df_.to_csv(output_file_name+'.tsv',sep='\t')

In [48]:
len(df)

161